In [4]:
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('seaborn-whitegrid')
import numpy as np

In [5]:
folder ='D:XXyourpathtodata/'
file='docwithtweets'#.csv
file2='docwithtweets2'#.csv

In [6]:
dfa = pd.read_csv(folder+file+'.csv')

In [7]:
dfa=dfa.drop_duplicates(subset=['tweet_id'])

In [8]:
len(dfa)

45178

# Tweets per day

In [9]:
dfa.tweet_created_at = pd.to_datetime(dfa['tweet_created_at'])

In [ ]:
print(dfa.dtypes)

In [11]:
dfa['just_date'] = dfa['tweet_created_at'].dt.date

In [ ]:
tweetsdate=dfa.groupby(["just_date"]).size().sort_values(ascending=False)
tweetsdate

In [13]:
cmap = ['blue','red','purple','black','green','cyan','orange', 'brown']

In [ ]:
#simple line plot (it saves directly)
line=tweetsdate.plot.line(color=cmap, rot=60, style='.-', use_index=True)
line.get_figure().savefig(folder+"tweetsperday.png")

# tweets per language

In [15]:
dfcounts=pd.DataFrame({'size' : dfa.groupby(dfa.lang).size()}).reset_index()
dfcountsdesc=dfcounts.sort_values('size', ascending=False)
dfcountsdesc['labels']=dfcountsdesc['lang']+ ' - '+dfcountsdesc['size'].astype('str')
languagetop=dfcountsdesc.head(10)

In [ ]:
languagetop

In [ ]:
#pie for the languages

ax=languagetop.plot(kind='pie', y='size', labeldistance=None,  figsize=(7,7))
ax.legend(mode='narrow',loc='best', bbox_to_anchor=(1, 1),
          title="Languages - number of tweets",
          labels=(languagetop.labels))
ax.set_ylabel("Size of the language", fontsize=14)
ax.set_title('Language of the Tweets',fontsize=18)
ax.get_figure().savefig(folder+"tweetsperlanguage.png")

# Tweets per language and day

In [18]:
langdate=pd.DataFrame({'size' : dfa.groupby(["lang", "just_date"]).size()}).reset_index()


In [19]:
langdate=langdate.sort_values('size', ascending=False)

In [20]:
langdate=langdate.rename(columns={"lang": "Language", "just_date":"Date", "size":"Number"}, errors="raise")

In [ ]:
langdate

In [22]:
#this is a list of the top languages (in languagetop); but the list can be taylored
languages= languagetop.lang.tolist()
#languages = ['en', 'es', 'ca', 'fr', 'ja','tr', 'und', 'de']
languages

['en', 'und', 'ja', 'qme', 'qht', 'kn', 'cy', 'es', 'ht', 'in']

In [23]:
#languages = ['en', 'es', 'ca', 'fr', 'ja','tr', 'und', 'de']
df3=langdate[langdate['Language'].isin(languages)]

In [24]:
df3 = df3.pivot(index='Date', columns='Language', values='Number')

In [25]:
cmap = ['blue','red','black','purple','green','cyan','orange', 'brown', 'yellow','grey']

In [ ]:
ax=df3.plot(kind='line', style='o-',   figsize=(10,10))
ax.legend(mode='narrow',loc='best', bbox_to_anchor=(1, 1),
          title="Languages",
          labels=(langdate.Language))
ax.set_ylabel("Number of tweets", fontsize=14)
ax.set_xlabel("Date", fontsize=14)
ax.set_title('Tweets per day and language',fontsize=18)
ax.get_figure().savefig(folder+"tweetsperlanguageday.png")

# User presence analysis (count-users)

In [27]:
dfusers= pd.DataFrame({'count' : dfa.groupby( [ "username"] ).size().sort_values(ascending=False)}).reset_index()
#top n users
dfusers.head(20)
#filter by number of tweets a user made
activeusers=dfusers[dfusers['count'] > 20]

In [ ]:
activeusers

In [ ]:
ax=activeusers.plot.bar(x='username', fontsize=10, y='count', figsize=(10,20))
plt.xticks(rotation=60)
x_offset = -0.5
y_offset = 2
ax.set_ylabel("Number of tweets", fontsize=14)
ax.set_xlabel("User", fontsize=14)
ax.set_title('Tweets per user',fontsize=18)
for p in ax.patches:
     #ax.annotate(np.round(p.get_height(),decimals=0)
    b = p.get_bbox()
    val = "{}".format(b.y1 + b.y0)      
    ax.annotate(np.round(p.get_height(),decimals=0),((b.x1 + b.x1)/2 + x_offset, b.y1 + y_offset))

# Count times specific words appear

In [31]:
wordlist=['Corbyn',
          'Boris Johnson',
          'aliens',
          'OVNIs',
          '#NHS']

In [32]:
#general wordcount without date:
wordcounts=[]
for i in wordlist:
    counts = dfa['text'].str.contains(str(i), na=False).sum()
    wordcounts.append(counts)

In [33]:
numwords=pd.DataFrame(
    {'word': wordlist,
     'count': wordcounts
    })

In [ ]:
numwords=numwords.sort_values(by=['count'], ascending=False)
numwords

In [ ]:
ax=numwords.plot.bar(x='word', fontsize=12,rot=60, y='count', color='red', width=0.5, figsize=(10,10))
#plt.xticks(rotation=90)
ax.set_ylabel("Number of appearances", fontsize=14)
ax.set_xlabel("Words", fontsize=14)
ax.set_title('Word appearances',fontsize=18)
x_offset = -0.5
y_offset = 2
for p in ax.patches:
    b = p.get_bbox()
    val = "{}".format(b.y1 + b.y0)      
    ax.annotate(str(p.get_height()), (p.get_x() , p.get_height()),  (b.x1 +0.2+ x_offset, b.y1 +30+ y_offset))
plt.savefig(folder+'keywordcount2.pdf')

In [77]:
# Keyword Count per day

In [81]:
#group by date (again) 
df1_grouped = dfa.groupby('just_date')


In [82]:
worddatecounts=[]
for i in wordlist:
    for group_date, df_group in df1_grouped:
        counts = df_group['text'].str.contains(str(i), na=False).sum()
        worddatecounts.append((str(i)+' // '+ str(group_date)+' -- '+ str(counts)))
        #print (str(i)+' '+ str(group_date)+' -- '+ str(counts))

In [ ]:
numwordsday=pd.DataFrame(
    {'word': worddatecounts})

numwordsday

In [ ]:
new = numwordsday["word"].str.split(" -- ", n = 1, expand = True)
numwordsday["rest"]= new[0]
numwordsday["other"]= new[1]
new = numwordsday["rest"].str.split(" // ", n = 1, expand = True)
numwordsday["key"]= new[0]
numwordsday["date"]= new[1]
numwordsday.drop(columns =["word"], inplace = True)
numwordsday.drop(columns =["rest"], inplace = True)
numwordsday=numwordsday.rename(columns={"key":"Word","date":"Date", "other": "Count"}, errors="raise")
numwordsday = numwordsday[['Word', 'Date', 'Count']]
numwordsday["Count"] = pd.to_numeric(numwordsday["Count"])
numwordsday['Date']= pd.to_datetime(numwordsday["Date"]).dt.date
numwordsday

In [ ]:
numwordsday['Date'].unique()

In [92]:
numwordsday1 = numwordsday.pivot(index='Date', columns='Word', values='Count')

In [ ]:
#cmap = ['blue','red','black','purple','green','cyan','orange', 'brown']
numwordsday1.plot.bar(
    stacked=True, 
    width=0.5,
    #color=cmap, 
    figsize=(20,10) 
    ,rot=60
    )
plt.savefig(folder+'keyworddatestack.pdf')

In [ ]:
numwordsday1.plot(
    #stacked=True, 
    #color=cmap, 
    figsize=(20,10) 
    ,rot=60
    , style='-o'
    )

plt.savefig(folder+'keyworddatesline.pdf')

# Clean Retweets

In [103]:

dfNORT=dfa[~dfa.text.str.contains("RT @")]

print("total of tweets = "+str(len(dfa))+" Without RT= "+ str(len(dfNORT)))

total of tweets = 45178 Without RT= 10472


In [ ]:
print(dfNORT.dtypes)